# Stock Price Forecasting AI Agent

This project is an AI-powered function-calling agent designed to forecast stock prices based on user-defined parameters. By leveraging the OpenAI API and advanced neural forecasting models, this agent enables users to receive accurate stock predictions for specified periods. The project is tailored for financial analysts, traders, and developers seeking quick, AI-driven insights into stock trends.

## Features

- **Customizable Forecast Period**: Users can specify the number of days (`prediction_days`) for the stock forecast, providing flexibility based on individual needs.
- **Function Calling with OpenAI API**: The agent uses OpenAI's function calling capabilities to interpret user prompts, extract relevant details like `stock_symbol` and `prediction_days`, and trigger the forecast function.
- **Advanced Forecasting Model**: Integrates the NeuralForecast framework with the NBEATS model for high-accuracy predictions using daily stock data from Yahoo Finance.
- **Automatic Data Retrieval**: Pulls historical stock data from Yahoo Finance, processes it, and prepares the dataset for model training and prediction.

## Workflow

1. **User Prompt**: The user specifies a stock symbol (e.g., "AAPL" or "MSFT") and a forecast duration in days.
2. **Agent Function Calling**: The agent, powered by OpenAI API, parses the prompt to identify the stock symbol and prediction days.
3. **Data Retrieval and Preparation**: Stock data is retrieved for the past year and preprocessed for the forecasting model.
4. **Forecast Generation**: Using NBEATS, the agent generates a stock price forecast, returning a dataframe with dates, predictions, and the ticker symbol.

## Example Usage

1. **Define User Request**: "Get stock price forecast for Qualcomm for 15 days."
2. **Agent Identifies Parameters**: The function schema parses `stock_symbol` as "QCOM" and `prediction_days` as 15.
3. **Generate and Output Forecast**: The agent calls the `get_stock_forecast` function, providing a 15-day forecast.

## Installation

1. Clone the repository:
   ```bash
   git clone <repository-url>
   cd <repository-name>

In [1]:
import yfinance as yf
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
import pandas as pd
from datetime import datetime, timedelta

/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-15 14:25:03,458	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-15 14:25:03,602	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Define Forecasting Function 

Use Deep Learning Model NBEATS from nitxla. 

In [ ]:
def get_stock_forecast(stock_symbol: str, prediction_days: int = 10) -> pd.DataFrame:
    """Forecasting function for any given ticker

    Args:
        stock_symbol (str): The stock symbol to forecast (e.g., 'AAPL', 'MSFT').
        prediction_days (int, optional): The prediction days. Defaults to 10.

    Returns:
        pd.DataFrame: Dataframe containing: 
            - ticker (string)
            - date (datetime.date)
            - prediction (float)
    """
    
    FREQUENCY='D'
    val_size = prediction_days 

    stock_data = yf.download(stock_symbol, period="1y", interval="1d")
    max_encoder_length = int(stock_data.shape[0]/10)
    
    stock_data = stock_data.reset_index().rename(columns={"Date": "ds", "Close": "y"})
    stock_data = stock_data[["ds", "y"]].copy()
    stock_data.columns = ["ds", "y"]
    stock_data['unique_id'] = stock_symbol

    models = [
        NBEATS(
            input_size=max_encoder_length,
            h=prediction_days,
            max_steps=max_encoder_length,
        ),
    ]

    nf = NeuralForecast(models=models, freq=FREQUENCY)
    nf.fit(df=stock_data, val_size=val_size, verbose=False)
    future_dates = nf.predict().reset_index()
    future_dates = future_dates.rename(columns={'unique_id':'ticker', 'ds':'date', 'NBEATS':'prediction'})

    return future_dates


In [ ]:
# Test the function
get_stock_forecast('TSLA', 20)

[*********************100%***********************]  1 of 1 completed
Seed set to 1
/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/common/_base_model.py:345: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
1.8 K     Non-trainable params
2.5 M     Total params
9.883     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 24: 100%|██████████| 1/1 [00:00<00:00, 26.40it/s, v_num=3, train_loss_step=16.30, train_loss_epoch=16.30, valid_loss=52.60]

`Trainer.fit` stopped: `max_steps=25` reached.


Epoch 24: 100%|██████████| 1/1 [00:00<00:00, 25.40it/s, v_num=3, train_loss_step=16.30, train_loss_epoch=16.30, valid_loss=52.60]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 147.36it/s]


/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


,ticker,date,prediction
0,TSLA,2024-11-15 00:00:00+00:00,303.157288
1,TSLA,2024-11-16 00:00:00+00:00,317.190094
2,TSLA,2024-11-17 00:00:00+00:00,315.939606
3,TSLA,2024-11-18 00:00:00+00:00,306.839996
4,TSLA,2024-11-19 00:00:00+00:00,305.162506
5,TSLA,2024-11-20 00:00:00+00:00,298.665222
6,TSLA,2024-11-21 00:00:00+00:00,305.226959
7,TSLA,2024-11-22 00:00:00+00:00,300.748383
8,TSLA,2024-11-23 00:00:00+00:00,297.048279
9,TSLA,2024-11-24 00:00:00+00:00,297.853699


## Create the Stock Forecasting Agent

In [21]:
import requests
import json
import os

def stock_forecast_agent(prompt: str)-> None:
    """Using OpenAI api and GPT4mini predict the stock price 
    of a company available in the Yahoo Finance API for any days in the futures.

    Args:
        prompt (str): the user prompt
    """
        
    api_key = os.environ['OPENAI_API_KEY']
    endpoint = "https://api.openai.com/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Define the function schema for OpenAI function calling
    function_schema = {
        "name": "get_stock_forecast",
        "description": "Get stock price forecast for a given number of days",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_symbol": {
                    "type": "string",
                    "description": "The stock symbol to forecast (e.g., 'AAPL', 'MSFT')."
                },
                "prediction_days": {
                    "type": "integer",
                    "description": "The number of days to forecast prices for."
                }
            },
            "required": ["stock_symbol"]
        }
    }

    messages = [
        {"role": "user", "content": prompt}
    ]

    data = {
        "model": "gpt-4o-mini",  
        "messages": messages,
        "functions": [function_schema],
        "function_call": "auto"
    }

    response = requests.post(endpoint, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        result = response.json()
        function_call = result["choices"][0].get("message", {}).get("function_call")

        if function_call:
            arguments = json.loads(function_call["arguments"])
            stock_symbol = arguments["stock_symbol"]
            
            prediction_days = arguments.get("prediction_days", 10)
            
            forecast = get_stock_forecast(stock_symbol, prediction_days)
            print(f"Forecast for {stock_symbol}:", forecast)
        else:
            print(result["choices"][0]["message"]["content"])
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")

In [29]:
prompt = "Get stock price forecast for NVIDIA for 15 days"
stock_forecast_agent(prompt=prompt)

[*********************100%***********************]  1 of 1 completed
Seed set to 1
/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/common/_base_model.py:345: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.5 M  | train
-------------------------------------------------------
2.5 M     Trainable params
1.2 K     Non-trainable params
2.5 M     Total params
9.830     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 24: 100%|██████████| 1/1 [00:00<00:00, 29.82it/s, v_num=19, train_loss_step=7.250, train_loss_epoch=7.250, valid_loss=4.470]

`Trainer.fit` stopped: `max_steps=25` reached.


Epoch 24: 100%|██████████| 1/1 [00:00<00:00, 28.56it/s, v_num=19, train_loss_step=7.250, train_loss_epoch=7.250, valid_loss=4.470]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 208.04it/s]
Forecast for NVDA:    ticker                      date  prediction
0    NVDA 2024-11-15 00:00:00+00:00  148.734985
1    NVDA 2024-11-16 00:00:00+00:00  149.379303
2    NVDA 2024-11-17 00:00:00+00:00  151.409531
3    NVDA 2024-11-18 00:00:00+00:00  149.521683
4    NVDA 2024-11-19 00:00:00+00:00  152.254272
5    NVDA 2024-11-20 00:00:00+00:00  153.005463
6    NVDA 2024-11-21 00:00:00+00:00  152.253403
7    NVDA 2024-11-22 00:00:00+00:00  153.665451
8    NVDA 2024-11-23 00:00:00+00:00  155.003418
9    NVDA 2024-11-24 00:00:00+00:00  154.867157
10   NVDA 2024-11-25 00:00:00+00:00  154.215607
11   NVDA 2024-11-26 00:00:00+00:00  156.888046
12   NVDA 2024-11-27 00:00:00+00:00  158.257233
13   NVDA 2024-11-28 00:00:00+00:00  157.735062
14   NVDA 2024-11-29 00:00:00+00:00  159.177231


/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
